In [28]:
import sagemaker
import boto3 
from sagemaker import Session 

sessao      = boto3.resource('s3')# sessao do s3, servico
ses         = sagemaker.Session() # sessai so sagemaker
rol = sagemaker.get_execution_role()


In [29]:
bucket= 'meunovo'
pasta = 'modelo/kmeans'
saida='s3://'+bucket+"/"+pasta
saida

's3://meunovo/modelo/kmeans'

In [30]:
from sagemaker import KMeans 

km = KMeans(role=rol,
             instance_count=1,
             instance_type='ml.c4.xlarge',
             output_path = saida,
             k=4)

In [35]:
from sklearn.preprocessing import StandardScaler
import numpy as np
base_salario = np.array([[20,1000],[27,1200],[21,2900],[37,1850],[46,900],
                        [53,950],[55,2000],[47,2100],[52,3000],[32,5900],
                        [39,4100],[41,5100],[39,7000],[48,5000],[48,6500]])

scaler_salario = StandardScaler() # preciso padronizar para fazer com que nao tenha problemas ao decidir valores mais importantes
base_salario = scaler_salario.fit_transform(base_salario)  
base_salario = base_salario.astype('float32')
base_salario

array([[-1.8796388 , -1.1141357 ],
       [-1.23255   , -1.0172544 ],
       [-1.7871976 , -0.19376273],
       [-0.3081375 , -0.7023899 ],
       [ 0.52383375, -1.1625764 ],
       [ 1.1709225 , -1.1383561 ],
       [ 1.355805  , -0.6297289 ],
       [ 0.616275  , -0.5812882 ],
       [ 1.0784813 , -0.14532205],
       [-0.7703438 ,  1.2594578 ],
       [-0.12325501,  0.38752547],
       [ 0.0616275 ,  0.8719323 ],
       [-0.12325501,  1.7923052 ],
       [ 0.7087163 ,  0.82349163],
       [ 0.7087163 ,  1.5501019 ]], dtype=float32)

In [36]:
km.fit(km.record_set(base_salario))

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-08-12 21:56:20 Starting - Starting the training job...
2022-08-12 21:56:48 Starting - Preparing the instances for trainingProfilerReport-1660341380: InProgress
.........
2022-08-12 21:58:18 Downloading - Downloading input data...
2022-08-12 21:58:48 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
[08/12/2022 21:59:27 INFO 139901186357056] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_profiler': 'false', '_kvstore': 'auto', '_log_level': 'info', '_num_gpus': 'auto', '_num_k

In [39]:
km_p = km.deploy(initial_instance_count=1,instance_type='ml.c4.xlarge')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


------!

In [58]:
test = np.array([[10 ,2000]])



In [59]:
rr = km_p.predict(test.astype('float32'))

In [60]:
rot = []
[r.label['closest_cluster'].float32_tensor.values[0] for r in rr]

[1.0]

In [61]:
km_p.delete_endpoint()